In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [106]:
# Data Manipulation
import pandas as pd
import numpy as np
from sklearn.neighbors import KNeighborsClassifier, NearestNeighbors, KNeighborsRegressor
from sklearn.metrics.pairwise import haversine_distances
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import RobustScaler, StandardScaler, MinMaxScaler, OneHotEncoder, LabelEncoder,OrdinalEncoder, MultiLabelBinarizer

from sklearn.pipeline import Pipeline
from sklearn.preprocessing import FunctionTransformer

# Load Dataset:

In [3]:
orgin_resturant_dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Yumspeak/cleaned_all_restaurants_dataset.csv')

Data Inspection:

In [4]:
orgin_resturant_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4510 entries, 0 to 4509
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   place_id          4510 non-null   object 
 1   name              4510 non-null   object 
 2   reviews           4510 non-null   int64  
 3   main_category     4510 non-null   object 
 4   categories        4510 non-null   object 
 5   main_rating       4510 non-null   float64
 6   address           4510 non-null   object 
 7   link              4510 non-null   object 
 8   review_photos     4510 non-null   object 
 9   coordinates       4510 non-null   object 
 10  latitude          4510 non-null   float64
 11  longtitude        4510 non-null   float64
 12  full_postal_code  4510 non-null   int64  
 13  postal_code       4510 non-null   int64  
 14  district_code     4510 non-null   int64  
 15  region            4510 non-null   object 
dtypes: float64(3), int64(4), object(9)
memory 

In [5]:
orgin_resturant_dataset.main_category.nunique()

156

## Model 1: Normal KNN with basic coordinate search only

Target & feature selection:

In [65]:
# 'main_rating': 'best', 'review': popular, 'n_neighbors': multi values
firstMod_feature_dataset = orgin_resturant_dataset[['latitude', 'longtitude']]
firstMod_target_dataset = orgin_resturant_dataset['main_category']

In [70]:
def norm_cord_search(test_coordinate, search_area = 3):
  KNN = KNeighborsClassifier(metric='haversine')
  KNN.fit(firstMod_feature_dataset.values, firstMod_target_dataset)
  neighbors = KNN.kneighbors(test_coordinate, n_neighbors=search_area, return_distance=False)
  return neighbors

Testing MODEL 1 ... ... ... 🛜

In [170]:
# Test coordinate
test_coordinate = [[1.284495, 103.852716]]
test_searchresult = 3

orgin_resturant_dataset.iloc[norm_cord_search(test_coordinate, test_searchresult)[0]]

,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude,full_postal_code,postal_code,district_code,region
255,ChIJCz-TrIoZ2jERBlG8g0v-5Nc,Taki Izakaya Bar,250,Modern izakaya,"['Modern izakaya restaurant', 'Bar', 'Izakaya ...",4.4,"16 Collyer Quay, #01-05A B, Singapore 049318",https://www.google.com/maps/place/Taki+Izakaya...,['https://lh5.googleusercontent.com/p/AF1QipOI...,"[1.284237, 103.8527103]",1.284237,103.852710,49318,4,1,City
3208,ChIJg4mRKrIZ2jERQleTjfSfIFU,Pasta e Formaggio(P&F) @ CIMB Plaza,172,Italian,"['Italian restaurant', 'Delivery Restaurant', ...",4.4,"16 Collyer Quay, #01-13 Centre, Singapore 049318",https://www.google.com/maps/place/Pasta+e+Form...,['https://lh5.googleusercontent.com/p/AF1QipOG...,"[1.2842345, 103.8524453]",1.284234,103.852445,49318,4,1,City
759,ChIJS0a4RocZ2jERFmEP0WYPG24,The Clifford Cafe,22,Cafe,['Cafe'],4.9,"50 Raffles Pl, #01-01 Singapore Land Tower, Si...",https://www.google.com/maps/place/The+Clifford...,['https://lh5.googleusercontent.com/p/AF1QipMA...,"[1.2847487, 103.8519923]",1.284749,103.851992,48623,4,1,City


# Model 2: KNN with rating search only

In [149]:
# Prepare feature dataset and target dataset
secondMod_feature_dataset = orgin_resturant_dataset[['latitude','longtitude']]
secondMod_target_dataset = orgin_resturant_dataset[['main_rating']]

In [133]:
# Encode 'main_category'
# Standard_Scaler = StandardScaler()
# secondMod_target_dataset['main_rating'] = Standard_Scaler.fit_transform(secondMod_target_dataset)

In [155]:
def norm_rating_cord_search(test_coordinate, search_area = 3):
  KNN_rating = KNeighborsRegressor(metric='haversine')
  KNN_rating.fit(secondMod_feature_dataset, secondMod_target_dataset)
  neighbors_distances_rating, neighbors_indices_rating = KNN_rating.kneighbors(test_coordinate, n_neighbors=search_area)
  return neighbors_indices_rating

In [169]:
test_coordinate = [[1.284495, 103.852716]]
test_searchresult = 5

# norm_rating_cord_search(test_coordinate)
modtwo_resturant_dataset = orgin_resturant_dataset.iloc[norm_rating_cord_search(test_coordinate, test_searchresult)[0]].sort_values(by='main_rating', ascending=False)
modtwo_resturant_dataset

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude,full_postal_code,postal_code,district_code,region
759,ChIJS0a4RocZ2jERFmEP0WYPG24,The Clifford Cafe,22,Cafe,['Cafe'],4.9,"50 Raffles Pl, #01-01 Singapore Land Tower, Si...",https://www.google.com/maps/place/The+Clifford...,['https://lh5.googleusercontent.com/p/AF1QipMA...,"[1.2847487, 103.8519923]",1.284749,103.851992,48623,4,1,City
255,ChIJCz-TrIoZ2jERBlG8g0v-5Nc,Taki Izakaya Bar,250,Modern izakaya,"['Modern izakaya restaurant', 'Bar', 'Izakaya ...",4.4,"16 Collyer Quay, #01-05A B, Singapore 049318",https://www.google.com/maps/place/Taki+Izakaya...,['https://lh5.googleusercontent.com/p/AF1QipOI...,"[1.284237, 103.8527103]",1.284237,103.852710,49318,4,1,City
3208,ChIJg4mRKrIZ2jERQleTjfSfIFU,Pasta e Formaggio(P&F) @ CIMB Plaza,172,Italian,"['Italian restaurant', 'Delivery Restaurant', ...",4.4,"16 Collyer Quay, #01-13 Centre, Singapore 049318",https://www.google.com/maps/place/Pasta+e+Form...,['https://lh5.googleusercontent.com/p/AF1QipOG...,"[1.2842345, 103.8524453]",1.284234,103.852445,49318,4,1,City
743,ChIJqft_S-MZ2jEREFW7mYRI_SM,Tanamera Coffee CIMB Plaza,269,Cafe,['Cafe'],4.4,"30 Raffles Pl, #01 - 12 / 13, Singapore 048622",https://www.google.com/maps/place/Tanamera+Cof...,['https://lh5.googleusercontent.com/p/AF1QipN3...,"[1.2841968, 103.8521065]",1.284197,103.852107,48622,4,1,City
2894,ChIJyaeiOAAZ2jER5WzrYzFK9OE,Marie's Lapis Café Express,18,',"['Restaurant', 'Singaporean restaurant']",4.4,"5 Raffles Pl, B1 - 33, Singapore 048618",https://www.google.com/maps/place/Marie's+Lapi...,['https://lh5.googleusercontent.com/p/AF1QipMu...,"[1.2844808, 103.8514561]",1.284481,103.851456,48618,4,1,City


# Model 3: KNN with reviews search only

In [165]:
# Prepare feature dataset and target dataset
secondMod_feature_dataset = orgin_resturant_dataset[['latitude','longtitude']]
secondMod_target_dataset = orgin_resturant_dataset[['reviews']]

In [166]:
def norm_rating_cord_search(test_coordinate, search_area = 3):
  KNN_rating = KNeighborsRegressor(metric='haversine')
  KNN_rating.fit(secondMod_feature_dataset, secondMod_target_dataset)
  neighbors_distances_rating, neighbors_indices_rating = KNN_rating.kneighbors(test_coordinate, n_neighbors=search_area)
  return neighbors_indices_rating

In [168]:
test_coordinate = [[1.284495, 103.852716]]
test_searchresult = 5

# norm_rating_cord_search(test_coordinate)
modtwo_resturant_dataset = orgin_resturant_dataset.iloc[norm_rating_cord_search(test_coordinate, test_searchresult)[0]].sort_values(by='reviews', ascending=False)
modtwo_resturant_dataset

/usr/local/lib/python3.10/dist-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but KNeighborsRegressor was fitted with feature names
  warnings.warn(


,place_id,name,reviews,main_category,categories,main_rating,address,link,review_photos,coordinates,latitude,longtitude,full_postal_code,postal_code,district_code,region
743,ChIJqft_S-MZ2jEREFW7mYRI_SM,Tanamera Coffee CIMB Plaza,269,Cafe,['Cafe'],4.4,"30 Raffles Pl, #01 - 12 / 13, Singapore 048622",https://www.google.com/maps/place/Tanamera+Cof...,['https://lh5.googleusercontent.com/p/AF1QipN3...,"[1.2841968, 103.8521065]",1.284197,103.852107,48622,4,1,City
255,ChIJCz-TrIoZ2jERBlG8g0v-5Nc,Taki Izakaya Bar,250,Modern izakaya,"['Modern izakaya restaurant', 'Bar', 'Izakaya ...",4.4,"16 Collyer Quay, #01-05A B, Singapore 049318",https://www.google.com/maps/place/Taki+Izakaya...,['https://lh5.googleusercontent.com/p/AF1QipOI...,"[1.284237, 103.8527103]",1.284237,103.852710,49318,4,1,City
3208,ChIJg4mRKrIZ2jERQleTjfSfIFU,Pasta e Formaggio(P&F) @ CIMB Plaza,172,Italian,"['Italian restaurant', 'Delivery Restaurant', ...",4.4,"16 Collyer Quay, #01-13 Centre, Singapore 049318",https://www.google.com/maps/place/Pasta+e+Form...,['https://lh5.googleusercontent.com/p/AF1QipOG...,"[1.2842345, 103.8524453]",1.284234,103.852445,49318,4,1,City
759,ChIJS0a4RocZ2jERFmEP0WYPG24,The Clifford Cafe,22,Cafe,['Cafe'],4.9,"50 Raffles Pl, #01-01 Singapore Land Tower, Si...",https://www.google.com/maps/place/The+Clifford...,['https://lh5.googleusercontent.com/p/AF1QipMA...,"[1.2847487, 103.8519923]",1.284749,103.851992,48623,4,1,City
2894,ChIJyaeiOAAZ2jER5WzrYzFK9OE,Marie's Lapis Café Express,18,',"['Restaurant', 'Singaporean restaurant']",4.4,"5 Raffles Pl, B1 - 33, Singapore 048618",https://www.google.com/maps/place/Marie's+Lapi...,['https://lh5.googleusercontent.com/p/AF1QipMu...,"[1.2844808, 103.8514561]",1.284481,103.851456,48618,4,1,City
